<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_vector_indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.pinecone.io/learn/series/faiss/vector-indexes/

https://www.youtube.com/watch?v=B7wmo_NImgM


矢量相似性搜索是搜索领域的游戏规则改变者。它使我们能够有效地搜索大量媒体，从 GIF 到文章，对于十亿以上大小的数据集，在亚秒级时间范围内具有令人难以置信的准确性。

高效搜索的关键要素之一是灵活性。为此，我们可以使用广泛的搜索索引——相似性搜索中不存在“一刀切”的索引。

然而，这种巨大的灵活性产生了一个问题——我们如何知道哪种尺寸适合我们的用例？

我们选择哪个索引？我们应该使用多个索引，还是一个就足够了？

本文将探讨一些最重要的指数（Flat、LSH、 HNSW和 IVF）的优缺点。我们将了解如何决定使用哪个以及每个索引中参数的影响。

# Indexes in Search

在讨论不同的可用索引之前，让我们先看看为什么我们关心相似性搜索，以及如何使用索引进行高效的相似性搜索。

## Why Similarity Search

让我们从本文相关性的最基本问题开始——为什么我们关心[相似性搜索](https://www.pinecone.io/learn/what-is-similarity-search/)？

相似性搜索可用于快速比较数据。给定一个查询（可以是任何格式——文本、音频、视频、GIF——只要你能想到的），我们可以使用相似性搜索来返回相关结果。

这对于跨行业的大量公司和应用程序来说至关重要。它用于识别基因组数据库中的相似基因、数据集的重复数据删除或每天搜索数十亿个结果以进行搜索查询。

搜索似乎是一个简单的过程——我们将一个项目与另一个项目进行比较。但是，当我们有数百万（甚至数十亿）个“其他”项目可供比较时，事情就开始变得棘手。

为了使搜索有用，它需要准确且快速。我们感兴趣的正是这种更有效的搜索。

## Indexes For Efficient Search

在向量相似性搜索中，我们使用索引来存储要搜索的数据的[向量表示](https://www.pinecone.io/learn/vector-embeddings/)。

通过[统计方法或机器学习](https://www.pinecone.io/learn/semantic-search/)，我们可以构建向量来编码有关原始数据的有用且有意义的信息。

![](https://cdn.sanity.io/images/vr8gru94/production/ef54ba69fe761ef25fb1b61fe6488c743f87862b-1280x720.png)
使用密集编码的向量，我们可以证明女人-女王语义关系和男人-国王语义关系是等价的。

我们获取这些“有意义的”向量并将它们存储在索引中以用于智能相似性搜索。

有很多可用的索引解决方案；其中一个特别是Faiss（[Facebook AI 相似性搜索](https://www.pinecone.io/learn/series/faiss/)）。我们将向量存储在 Faiss 中，并使用“查询”向量查询新的 Faiss 索引。将此查询向量与其他索引向量进行比较，以找到最接近的匹配项 - 通常使用欧几里得 (L2) 或内积 (IP) 指标。

因此，介绍了相似性搜索的原因和方式。但 Faiss 的意义何在——以及如何选择正确的索引？


## Faiss And Indexes
Faiss 具有许多不同的索引类型 - 其中许多可以混合和匹配以生成多层索引。

我们将重点关注一些优先考虑搜索速度、质量或索引内存的索引。

现在，我们使用哪一个索引很大程度上取决于我们的用例。我们必须考虑数据集大小、搜索频率或搜索质量与搜索速度等因素。

## Flat And Accurate
![](https://cdn.sanity.io/images/vr8gru94/production/ca40b2c0017011212a1cf1a98d0fa76f5a96c260-1280x720.png)

Flat indexes come with perfect search-quality at the cost of slow search speeds. Memory utilization of flat indexes is reasonable.

我们应该关注的第一个索引是最简单的——扁平索引(flat index)。

平坦索引是“平坦”的，因为我们不修改输入它们的向量。

因为我们的向量没有近似或聚类 - 这些索引产生最准确的结果。我们拥有完美的搜索质量，但这是以大量搜索时间为代价的。

使用平面索引，我们引入查询向量xq并将其与索引中的每个其他全尺寸向量进行比较 - 计算到每个向量的距离。

![](https://cdn.sanity.io/images/vr8gru94/production/c39faec676b5276b00e3149115534ca1ddc3e16c-1920x1080.png)
使用平面索引，我们将搜索查询 xq 与索引中的每个其他向量进行比较。

计算完所有这些距离后，我们将返回其中最接近的 k 个距离作为最接近的匹配项。k 最近邻 (kNN) 搜索。

![](https://cdn.sanity.io/images/vr8gru94/production/9f4a2a609027c2ae09ef0430ab7f34d85c7e5c0a-700x393.png)
计算完所有距离后，我们将返回 k 个最近的向量。

## When To Use

那么我们什么时候应该使用扁平索引呢？好吧，当搜索质量无疑是一个高优先级时，搜索速度就不那么重要了。

对于较小的数据集，搜索速度可能是一个无关紧要的因素——尤其是在使用更强大的硬件时。

![](https://cdn.sanity.io/images/vr8gru94/production/fc786da280703d68bee6b4ff552bbc0d5b799311-700x393.png)
在 M1 芯片上使用 faiss-cpu 的欧几里得 (L2) 和内积 (IP) 平面索引搜索时间。两者都使用向量维度 100。IndexFlatIP 比 IndexFlatL2 稍快。

上图展示了 M1 芯片上的 Faiss CPU 速度。Faiss 经过优化，与 Linux 上支持 CUDA 的 GPU 配合使用时，可以以显着更高的速度在 GPU 上运行，从而显着缩短搜索时间。

简而言之，在以下情况下使用平面索引：(比如指纹(公安系统)，人脸认证场景(购票人乘车))

搜索质量是非常重要的。

当使用小索引（<10K）时，搜索时间并不重要。

## Implementing a Flat Index
要初始化平面索引，我们需要数据 Faiss 和两个平面索引之一 - IndexFlatL2（如果使用欧几里得/L2 距离）或IndexFlatIP（如果使用内积距离）。

首先，我们需要数据。我们将使用[Sift1M 数据集](http://corpus-texmex.irisa.fr/)，我们可以使用以下命令将其下载并加载到笔记本中：










In [1]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [2]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [3]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [5]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [6]:
xq.shape

(1, 128)

In [7]:
xb.shape

(1000000, 128)

In [8]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

现在我们可以使用两个平面索引之一来索引新数据。我们发现 IndexFlatIP 稍微快一点，所以让我们使用它。



In [10]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 0s (2,454 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

In [13]:
d = 128  # dimensionality of Sift1M data
k = 10  # number of nearest neighbors to return

import faiss
index = faiss.IndexFlatIP(d)
index.add(xb)
D, I = index.search(xq, k)
print(I,D)

[[932085 934876 561813 708177 706771 695756 435345 701258 872728 455537]] [[230843. 230600. 228242. 225288. 225144. 224256. 224214. 223408. 222827.
  222341.]]


对于平面索引，这就是我们需要做的一切 - 无需训练（因为在存储没有转换或聚类的向量时，我们没有可优化的参数）。



## Balancing Search Time
扁平索引非常准确，但速度非常慢。在相似性搜索中，搜索速度和搜索质量（准确性）之间总是存在权衡。

我们必须做的是确定我们的用例最佳点在哪里。对于扁平索引，我们在这里：

![](https://cdn.sanity.io/images/vr8gru94/production/bef5666ef59da2a1d2b45afd19295df1b79b3770-700x183.png)

*扁平索引的搜索质量为 100%，搜索速度为 0%*。


这里我们有完全未优化的搜索速度，这将适合许多较小的索引用例 - 或搜索时间无关的场景。但是，其他用例需要更好地平衡速度和质量。

那么，我们怎样才能让我们的搜索更快呢？有两种主要方法：

1. 减小向量大小 - 通过降维或减少表示向量值的位数。

2. 缩小搜索范围——我们可以通过根据某些属性、相似性或距离将向量聚类或组织成树结构来实现这一点——并将我们的搜索限制在最近的集群或过滤最相似的分支。


使用这两种方法中的任何一种都意味着我们不再执行详尽的最近邻搜索，而是执行近似最近邻（ANN）搜索——因为我们不再搜索整个全分辨率数据集。

因此，我们产生的是一个更平衡的组合，优先考虑搜索速度和搜索时间：

![](https://cdn.sanity.io/images/vr8gru94/production/374584f480530b269a8ac0d860e0bce98c3612c7-700x183.png)

*通常，我们希望搜索速度和搜索质量更加平衡*。

## 局部敏感哈希（LSH-Locality Sensitive Hashing）
![](https://cdn.sanity.io/images/vr8gru94/production/02b1b08c1d5eac76f6472454c702cc70edd9d9da-1280x720.png)

*LSH — 各种性能严重依赖于参数集。良好的质量会导致搜索速度变慢，而快速的搜索会导致质量较差。高维数据的性能较差。条形的“半填充”部分表示修改索引参数时遇到的性能范围*。

局部敏感哈希 (LSH) 的工作原理是通过哈希函数处理每个向量，将向量分组到存储桶中，从而最大化哈希冲突，而不是像通常的哈希函数那样最小化冲突。

这意味着什么？假设我们有一个 Python 字典(dict)。当我们在字典中创建新的键值对时，我们使用哈希函数对键进行哈希处理。键的哈希值决定了我们存储其各自值的“桶”：
![](https://cdn.sanity.io/images/vr8gru94/production/eecf5ed2893571383d618650d40810140dd987f6-700x393.png)

*类似字典的对象的典型哈希函数将尝试最小化哈希冲突，旨在为每个存储桶仅分配一个值*。

Python 字典是哈希表的一个示例，它使用典型的哈希函数来最大限度地减少哈希冲突，即两个不同对象（键）产生相同哈希的哈希冲突。

在我们的字典中，我们希望避免这些冲突，因为这意味着我们会将多个对象映射到单个键 - 但对于 LSH，我们希望最大化散列冲突。

为什么我们要最大化碰撞？好吧，对于搜索，我们使用 LSH 将相似的对象分组在一起。当我们引入一个新的查询对象（或向量）时，我们的 LSH 算法可用于查找最接近的匹配组：

![](https://cdn.sanity.io/images/vr8gru94/production/4c72c87199038e6875710893c81eb0291019756e-700x393.png)

*我们的 LSH 哈希函数尝试最大化哈希冲突，生成向量分组。*

## Implementing LSH
在 Faiss 中实施我们的 LSH 索引很容易。我们使用向量维度 d 和 nbits 参数初始化一个 IndexLSH 对象，并像这样添加向量：






In [15]:
nbits = d*4  # resolution of bucketed vectors
# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.add(xb)
# and search
D, I = index.search(xq, k)
print(I,D)

[[435345 931632 708177 813701 934876 455537 932085 561813 248185 361496]] [[75. 75. 76. 76. 76. 77. 77. 78. 79. 81.]]


我们的nbits参数指的是散列向量的“分辨率”。较高的值意味着更高的准确性，但代价是更多的内存和更慢的搜索速度。

![](https://cdn.sanity.io/images/vr8gru94/production/31da38bd71537369c2320fd9def08f9c4ecb17df-700x393.png)

*IndexLSH 的召回分数，d 为 128。请注意，为了获得更高的召回性能，我们需要大幅增加 num_bits 值。对于 90% 的召回率，我们使用 64d，即 64128 = 8192。*

我们的基准IndexFlatIP指数是我们 100% 的召回率性能，使用IndexLSH，我们可以使用非常高的nbits值实现 90% 的召回率。

这是一个强有力的结果——如果我们改善搜索时间，90% 的性能肯定是对性能的合理牺牲。

然而，当使用较大的d值时，LSH 对维数灾难高度敏感，我们还需要增加nbits以保持搜索质量。

因此，随着原始向量维数d的增加，我们存储的向量变得越来越大。这很快就会导致搜索时间过长：

![](https://cdn.sanity.io/images/vr8gru94/production/5d659ac67f9eddf2f511baa4d337bb47af0f5e3c-700x393.png)

*与平坦 IP 索引相比，具有不同 nbits 值的 IndexLSH 的搜索时间。*

这由我们的索引内存大小反映：

![](https://cdn.sanity.io/images/vr8gru94/production/a1642ab25684d49c0135e5a979fcb056e1311303-700x393.png)

*与固定 IP 索引相比，具有不同 nbits 值的 IndexLSH 的索引大小。*

因此，如果向量维数较大（128已经太大），则IndexLSH不适合。相反，它最适合低维向量和小索引。

如果我们发现自己有大的d值或大的索引 - 我们完全避免 LSH，而是专注于我们的下一个索引 HNSW。

## **分层可导航小世界图 (HNSW-Hierarchical Navigable Small World Graphs)**

![](https://cdn.sanity.io/images/vr8gru94/production/9f9235729775b53415bb17c4ffc6d387a64d4cf9-1280x720.png)

*HNSW — 出色的搜索质量、良好的搜索速度，但索引大小很大。条形的“半填充”部分表示修改索引参数时遇到的性能范围。*

分层可导航小世界 (HNSW) 图是搜索领域的另一个最新发展。基于 HNSW 的 ANNS 始终名列表现最高的指数 [1]。

HNSW 是可导航小世界 (NSW) 图的进一步改编，其中 NSW 图是包含通过边连接到最近邻居的顶点的图结构。

“NSW”部分是因为这些图中的顶点到图中所有其他顶点的平均路径长度都非常短 - 尽管没有直接连接。

以 Facebook 为例——在 2016 年，我们可以将每个用户（一个顶点）连接到他们的 Facebook 好友（他们最近的邻居）。尽管有 1.59B 活跃用户，但从一个用户遍历该图到另一个用户所需的平均步数（或跳数）仅为 3.57 [2]。

![](https://cdn.sanity.io/images/vr8gru94/production/dadef98124820d9183c8fb58912f8443f63a412c-700x422.png)

*NSW 图的可视化。请注意，每个点与另一个点的距离不超过四跳。*

Facebook 只是庞大网络中高连接性的一个例子 - NSW 图

在较高层次上，HNSW 图是通过获取 NSW 图并将其分解为多个层来构建的。每个增量层都消除了顶点之间的中间连接。


![](https://cdn.sanity.io/images/vr8gru94/production/00b3255c6332c094eb6bbca5d264615b16c34562-700x393.png)

*通过 HNSW，我们将网络分成几个层，在搜索过程中遍历这些层。*

对于具有更高维度的更大数据集 - HNSW 图是我们可以使用的一些性能最佳的索引。我们将单独使用 HNSW 索引，但通过分层其他量化步骤，我们可以进一步缩短搜索时间。

## HNSW Implementation

要在 Faiss 中构建和搜索平面(flat) HNSW 索引，我们需要的是IndexHNSWFlat：



In [16]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search
# add data to index
index.add(xb)

# search as usual
D, I = index.search(xq, k)
print(I,D)

[[932085 934876 561813 708177 706771 695756 435345 701258 455537 872728]] [[54229. 55091. 59531. 65260. 65697. 67010. 68247. 69844. 71441. 71861.]]


在这里，我们有三个关键参数来修改索引性能。

1. M——每个顶点将连接到的最近邻居的数量。

2. efSearch — 搜索期间将在层之间探索多少个入口点。

3. efConstruction — 构建索引时将探索多少个入口点。


可以增加每个参数以提高搜索质量：

![](https://cdn.sanity.io/images/vr8gru94/production/41d147152ae1d36d44d6dca4eae5bee95a1e4bff-1830x720.png)

*调用 Sift1M 数据集上不同 efConstruction、efSearch 和 M 值的值。*

M和efSearch对搜索时间有更大的影响 — efConstruction主要增加了索引构建时间（意味着较慢的index.add），但在M值较高和查询量较高时，我们确实看到efConstruction对搜索时间也有影响。

![](https://cdn.sanity.io/images/vr8gru94/production/e548534b6fe98462c4327e03904d97307e8bf1a8-1280x720.png)

*在完整 Sift1M 数据集上不同 M 和 efSearch 值的搜索时间。*

HNSW 以非常快的搜索速度为我们提供了出色的搜索质量 - 但总有一个问题 - HNSW 索引占用大量内存。对 Sift1M 数据集使用M值128需要 1.6GB 以上的内存。

![](https://cdn.sanity.io/images/vr8gru94/production/8b5b6604756e09e36cfae89ba0805c6b155640e4-700x393.png)

*Sift1M 数据集上不同 M 值的索引内存使用情况。*

但是，我们可以增加其他两个参数 - efSearch和efConstruction，而不影响索引内存占用。

因此，在 RAM 不是限制因素的情况下，HNSW 是一个很好的平衡指标，我们可以通过增加三个参数来推动更多地关注质量。

![](https://cdn.sanity.io/images/vr8gru94/production/dec2e681b4cedf7700431af4627fd4723e36012d-700x462.png)

*我们可以使用较低的参数集来平衡优先级稍快的搜索速度和良好的搜索质量，或者我们使用较高的参数集来平衡稍慢的搜索速度和较高的搜索质量。*

这就是 Faiss 中的 HNSW——一个极其强大且高效的索引。现在让我们进入最终索引——IVF。

## 倒排文件索引 (Inverted File Index)

![](https://cdn.sanity.io/images/vr8gru94/production/e38ad40fe8c2573c71a5ac4ac92cd58b11833480-1280x720.png)

*IVF — 出色的搜索质量、良好的搜索速度和合理的内存使用。条形的“半填充”部分表示修改索引参数时遇到的性能范围。*

倒排文件索引 (IVF) 包括通过聚类缩小搜索范围。它是一个非常受欢迎的索引，因为它易于使用、搜索质量高、搜索速度合理。

它基于 Voronoi 图的概念——也称为 Dirichlet 曲面细分（一个更酷的名字）。

为了理解 Voronoi 图，我们需要想象将高维向量放入 2D 空间中。然后，我们在 2D 空间中放置一些额外的点，这些点将成为我们的“簇(cluster)”（在本例中为 Voronoi 单元）质心(centroids)。

然后我们从每个质心向外延伸相等的半径。在某些时候，每个细胞圆的圆周都会与另一个细胞发生碰撞——形成我们的细胞边缘：

![](https://d33wubrfki0l68.cloudfront.net/4143695359a899afc205c9270035d984dea27607/56a39/images/similarity-search-indexes2.mp4)

*如何构建沃罗诺伊单元——在这里，我们有三个质心，从而产生三个沃罗诺伊单元。然后将给定单元内的每个数据点分配给相应的质心。*

现在，每个数据点都将包含在一个单元格中，并分配给相应的质心。

就像我们的其他索引一样，我们引入一个查询向量xq - 该查询向量必须落在我们的一个单元格内，此时我们将搜索范围限制在该单个单元格内。

但是，如果我们的查询向量落在单元格边缘附近，就会出现问题——它最近的其他数据点很可能包含在相邻单元格内。我们称之为边缘问题：

![](https://cdn.sanity.io/images/vr8gru94/production/5a44e6ded9916f127a76d45708baa20e02802574-700x437.png)

*我们的查询向量 xq 落在洋红色单元格的边缘。尽管更接近青色单元格中的数据点，但如果 nprobe == 1，我们不会比较这些数据点 - 因为这意味着我们将搜索范围仅限于洋红色单元格。*

现在，为了缓解这个问题并提高搜索质量，我们可以做的是增加称为nprobe值的索引参数。使用nprobe我们可以设置要搜索的单元格数量。

![](https://cdn.sanity.io/images/vr8gru94/production/f32a71b57eefa87ef461bb3412f9fc21bbd46514-2020x1270.png)
![](https://cdn.sanity.io/images/vr8gru94/production/e68c5241fd2726395449721f5414bc21b038f615-2020x1270.png)

*增加 nprobe 会增加我们的搜索范围。*

## IVF Implementation

要实现 IVF 索引并在搜索中使用它，我们可以使用IndexIVFFlat：








In [18]:
nlist = 128  # number of cells/clusters to partition data into

quantizer = faiss.IndexFlatIP(d)  # how the vectors will be stored/compared
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.train(xb)  # we must train the index to cluster into cells
index.add(xb)

index.nprobe = 8  # set how many of nearest cells to search
D, I = index.search(xq, k)
print(I,D)

[[932085 934876 561813 708177 706771 695756 435345 701258 455537 872728]] [[54229. 55091. 59531. 65260. 65697. 67010. 68247. 69844. 71441. 71861.]]


这里有两个参数我们可以调整。

1. nprobe — 要搜索的单元格数量

2. nlist — 要创建的单元格数量


较高的nlist意味着我们必须将向量与更多的质心向量进行比较 - 但在选择最近的质心单元进行搜索后，每个单元内的向量将会更少。因此，增加 nlist来优先考虑搜索速度。

至于nprobe，我们发现相反的情况。增加 nprobe会增加搜索范围 - 从而优先考虑搜索质量。

![](https://cdn.sanity.io/images/vr8gru94/production/8432b8975f19db1a0bc3fb4ced7f62ab7ab783f4-700x568.png)

*使用不同 nprobe 和 nlist 值的 IVF 搜索时间和召回率。*

就内存而言，IndexIVFFlat相当高效——修改nprobe不会影响这一点。nlist对内存使用的影响也很小——nlist越高意味着内存需求稍大。

![](https://cdn.sanity.io/images/vr8gru94/production/22d3d611b1d7096ba36e088329770f7ffeacec70-700x393.png)

*索引的内存使用情况仅受 nlist 参数影响。然而，对于我们的 Sift1M 数据集，索引大小仅发生很小的变化。*

因此，我们必须在nprobe更高的搜索质量和nlist更快的搜索速度之间做出选择。

----

我们在本文中介绍了很多内容，因此让我们快速总结一下每个索引的内存、速度和搜索质量性能。



|      Index      | Memory (MB) | Query Time (ms) |   Recall   |                            Notes                             |
| :-------------: | :---------: | :-------------: | :--------: | :----------------------------------------------------------: |
| Flat (L2 or IP) |    ~500     |       ~18       |    1.0     |  Good for small datasets or where query time is irrelevant   |
|       LSH       |  20 - 600   |    1.7 - 30     | 0.4 - 0.85 |       Best for low dimensional data, or small datasets       |
|      HNSW       | 600 - 1600  |    0.6 - 2.1    | 0.5 - 0.95 |  Very good for quality, high speed, but large memory usage   |
|       IVF       |    ~520     |      1 - 9      | 0.7 - 0.95 | Good scalable option. High-quality, at reasonable speed and memory usage |

此处显示的值是在 Sift1M 数据集上测试每个索引期间经历的值范围，代表每个参数设置下 20 次测试的平均结果。本文中测试的不切实际的参数设置（过低/过高）已被排除在该范围之外。

Sift1M 数据集的维数 d 为 128，数据集大小为 1M。上述值是查询单个向量的结果 - 返回 k == 10 个最接近的匹配项。

## 使用的硬件

* M1芯片，8核CPU
* 8GB统一内存

因此，我们有四个索引，具有同样有效的优点和缺点 - 取决于用例。希望通过本文，您现在可以更好地决定哪些索引最适合您自己的用例。

除了这些指标之外，还可以通过添加其他量化或压缩步骤来进一步提高内存使用率和搜索速度——但这是另一篇文章的内容了。

## 参考
[1] E Bernhardsson，[ANN 基准存储库](https://github.com/erikbern/ann-benchmarks)，Github

[2] S Edunov 等人，[三度半分离](https://research.fb.com/blog/2016/02/three-and-a-half-degrees-of-separation/)(2016)，Facebook 研究

除特别说明外，所有图片均由作者提供



